In [1]:
import keras
import pandas as pd
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.models import Sequential, Model
from keras.layers import MaxPooling1D, Dropout, Dense, Flatten, Reshape, Input, Concatenate
from keras.layers import Convolution1D as Conv1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import sklearn.metrics as skm
import tensorflow as tf

print('Keras version: ',keras.__version__)

Using TensorFlow backend.
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

Keras version:  2.3.0


In [17]:
def create_model(file, origen=True):
    convmodel = Sequential()
    convmodel.add(Conv1D(filters=10, kernel_size=3, activation='relu', input_shape=(7,1))) # los vectores tienen 7 elementos cada uno
    convmodel.add(Conv1D(filters=10, kernel_size=2, activation='relu'))
    convmodel.add(Dropout(0.1))
    convmodel.add(MaxPooling1D(pool_size=2))
    
    #define two different inputs
    input1 = Input((7, 1))
    input2 = Input((7, 1))
    input3 = Input((7, 1))
    input4 = Input((7, 1))
    input5 = Input((7, 1))
    input6 = Input((15, ))

    #use the model to get three different outputs (the same model)
    out1 = convmodel(input1)
    out2 = convmodel(input2)
    out3 = convmodel(input3)
    out4 = convmodel(input4)
    out5 = convmodel(input5)

    #concatenate the outputs and add the final part of your model: 
    out = Concatenate()([out1,out2,out3,out4,out5])
    out = Flatten()(out)
    out = Concatenate()([out,input6])  
    out = Dense(50, activation='relu')(out)
    out = Dense(50, activation='relu')(out)
    
    outputs=5
    if origen==False:
        outputs=4
    
    out = Dense(outputs, activation='softmax')(out)   

    #create the model taking 2 inputs with one output
    model = Model([input1,input2,input3,input4,input5,input6],out)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.load_weights(file)
    return model

In [7]:
def compactState(yard):
    sort = []
    for stack in yard:
      for container in stack:
        if not container in sort:
          sort.append(container)
    sort = sorted(sort)
    maxValue = 0
    for i in range(len(yard)):
      for j in range(len(yard[i])):
        yard[i][j] = sort.index(yard[i][j]) + 1
        if yard[i][j] > maxValue:
          maxValue = yard[i][j]
    return yard

def elevateState(yard, h, max_item):
    for stack in yard:
      while len(stack) < h:
        #stack.insert(0,max_item)
        stack.insert(0,1)
    return yard

def flattenState(state):
    flatten = []
    for lista in state:
        for item in lista:
            flatten.append(item)
    return flatten

def normalize(state,max_item):
    return np.array(state)/max_item


def getStackValues(yard): #sorted stacks?
    values = []
    for stack in yard:
        flag = False
        cont = 0
        for i in range(len(stack)):
            if i==0:
                cont += 1
            else:
                if stack[i] <= stack[i-1]:
                    cont += 1
                else: break
        values.append(cont)
    return values

def getStackValues_old(yard):
    values = []
    for stack in yard:
        flag = False
        cont = 0
        for i in range(len(stack)):
            if flag:
                cont += 1
            else:
                if i == 0:
                    continue
                if stack[i] > stack[i-1]:
                    cont += 1
                    flag = True
        values.append(cont)
    return values

def getStackLen(yard):
    lens = []
    for stack in yard:
        lens.append(len(stack))
    return lens

def getTopStacks(yard,max_item):
    tops = []
    for stack in yard:
        if len(stack) != 0:
            tops.append(stack[len(stack)-1])
        else:
            tops.append(max_item)
    return tops


In [30]:
import copy
def generate_ann_state_old(yard,H):
    yard=copy.deepcopy(yard)
    max_item = max(set().union(*yard))
    #max_item = max([max(stack) for stack in yard])
    stackValues=getStackValues_old(yard) #well-placed
    stacksLen = getStackLen(yard)
    topStacks = getTopStacks(yard,max_item)
    yard=elevateState(yard,H, max_item)
    yard=compactState(yard)
    yard=flattenState(yard)
    yard=normalize(yard,max_item)
    state=yard
    state=np.append(state,normalize(stackValues,max_item))
    state=np.append(state,normalize(stacksLen,max_item))
    state=np.append(state,normalize(topStacks,max_item))
    return state

In [12]:
def predict_origin(layout,model):
    X_test=generate_ann_state_old(layout.stacks,7)
    X1=X_test[0:7]
    X2=X_test[7:14]
    X3=X_test[14:21]
    X4=X_test[21:28]
    X5=X_test[28:35]
    X6=X_test[35:50]
    X1.shape=(1,7,1)
    X2.shape=(1,7,1)
    X3.shape=(1,7,1)
    X4.shape=(1,7,1)
    X5.shape=(1,7,1)
    X6.shape=(1,15)
    return np.argmax(model.predict([X1,X2,X3,X4,X5,X6]))

def predict_destination(layout,model,origin):
    X_test=generate_ann_state_old(layout.stacks,7)
    X1=X_test[0:7]
    X2=X_test[7:14]
    X3=X_test[14:21]
    X4=X_test[21:28]
    X5=X_test[28:35]
    X6=X_test[35:50]
    X1.shape=(1,7,1)
    X2.shape=(1,7,1)
    X3.shape=(1,7,1)
    X4.shape=(1,7,1)
    X5.shape=(1,7,1)
    X6.shape=(1,15)
    input = [X1,X2,X3,X4,X5,X6]
    aux = input[0] 
    input[0] = input[origin] 
    input[origin] = aux
    dest =  np.argmax(model.predict([X1,X2,X3,X4,X5,X6]))+1
    if dest==origin: return 0
    return dest

In [18]:
model_origen = create_model('model_origen.h5')
model_destino = create_model('model_destino.h5',origen=False)

In [111]:
import Pixie as px
layout = px.read("5-5/data5-5-10.dat")
print(layout.stacks)

origin = predict_origin(layout,model_origen)
destination = predict_destination(layout,model_destino,origin)
layout.move(origin,destination)
layout.stacks

[[11, 2, 3, 14, 19], [18, 17, 4, 9, 23], [24, 6, 25, 7, 21], [15, 1, 13, 22, 10], [12, 20, 5, 8, 16]]


[[11, 2, 3, 14, 19],
 [18, 17, 4, 9],
 [24, 6, 25, 7, 21],
 [15, 1, 13, 22, 10, 23],
 [12, 20, 5, 8, 16]]

In [118]:
origin = predict_origin(layout,model_origen)
destination = predict_destination(layout,model_destino,origin)
layout.move(origin,destination)
layout.stacks

IndexError: list index out of range

In [119]:
layout.stacks

[[],
 [18, 17, 4, 14, 3, 2, 11],
 [24, 6, 25, 7, 21],
 [15, 1, 13, 22, 10, 23, 19, 9],
 [12, 20, 5, 8, 16]]